# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-db3004a505-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

In [2]:
#return properties for Lebron
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [3]:
#return Lebron's occupation
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


Final query for this task

In [2]:
#Identify the BGP for basketball player.
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


## Task 2
Identify the BGP for basketball.

In [5]:
#return properties for basketball player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3665646 ?p ?o .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.w

In [6]:
#return sport for a basketball player
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q3665646 wdt:P641 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


Final query for this task

In [3]:
#Identify the BGP for basketball.
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q3665646 wdt:P641 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


## Task 3
Identify the BGP for basketball team.

In [8]:
#return more properties for basketball player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3665646 ?p ?o .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.w

In [9]:
#return what a basketball player is part of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q3665646 wdt:P361 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


Final query for this task

In [4]:
#Identify the BGP for basketball team.
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q3665646 wdt:P361 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [11]:
#return properties for basketball team
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q13393265 ?p ?o .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


In [12]:
#return objects linked by those properties
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q13393265 ?p ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q51747567'), ('name', 'squad')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
[('o', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('o', 'http://www.wikidata.org/entity/Q5137571'), ('name', 'basketball coach')]
[('o', 'http://www.wikidata.org/entity/Q847017'), ('name', 'sports club')]
[('o', 'http://www.wikidata.org/entity/Q12973014'), ('name', 'sports team')]
[('o', 'http://www.wikidata.org/entity/Q5648512'), ('name', 'Template:Infobox basketball club')]
[('o', 'http://www.wikidata.org/entity/Q5647277'), ('name', 'Category:Basketball teams')]
8


In [13]:
#return properties for squad
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q51747567 ?p ?o .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
4


In [14]:
#return properties and objects for squad
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q51747567 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q18536342'), ('oName', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q327245'), ('oName', 'team')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q847017'), ('oName', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q2066131'), ('oName', 'athlete')]
5


In [15]:
#return properties for team
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q327245 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q43229'), ('oName', 'organization')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q71690273'), ('oName', 'nonteam')]
4


In [16]:
#return properties for Template:Infobox basketball club
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q5648512 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q19887878'), ('oName', 'Wikimedia infobox template')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q13393265'), ('oName', 'basketball team')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13361807'), ('oName', 'Template:Infobox sport governing body')]
3


In [17]:
#return properties and objects for sports club 
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q847017 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4438121'), ('oName', 'sports organization')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q19467997'), ('oName', 'Template:Infobox athletics club')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q988108'), ('oName', 'club')]
[('p

In [18]:
#return properties and objects for squad 
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q51747567 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q18536342'), ('oName', 'competitive player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q327245'), ('oName', 'team')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('o', 'http://www.wikidata.org/entity/Q847017'), ('oName', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q2066131'), ('oName', 'athlete')]
5


In [19]:
#return properties and objects for team
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q327245 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q43229'), ('oName', 'organization')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q71690273'), ('oName', 'nonteam')]
4


In [20]:
#return properties and objects for sports club 
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q847017 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4438121'), ('oName', 'sports organization')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q19467997'), ('oName', 'Template:Infobox athletics club')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q988108'), ('oName', 'club')]
[('p

In [21]:
#return properties and objects for club
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   # bind something
   wd:Q988108 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12047392'), ('oName', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q43229'), ('oName', 'organization')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q622425'), ('oName', 'nightclub')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q48204'), ('oName', 'voluntary organization')]
[('p', 'http://www.wikidata.org/prop/

In [22]:
#return properties and objects for organization
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q43229 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3500'), ('oName', 'Ringgold ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P213'), ('oName', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oName', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3549'), ('oName', 'Australian Company Number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1278'), ('oName', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1320'), ('oName', 'Open

In [23]:
#return properties for Kevin Durant
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q29545 ?p ?o .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1618

In [24]:
#return Kevin Durant league
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29545 wdt:P118 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q94861615'), ('name', "NCAA Division I men's basketball")]
[('o', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
2


In [25]:
#return what NBA is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q155223 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q18536323'), ('name', 'basketball league')]
[('o', 'http://www.wikidata.org/entity/Q15986354'), ('name', 'basketball association')]
[('o', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('o', 'http://www.wikidata.org/entity/Q7094076'), ('name', 'online database')]
4


In [26]:
#return properties for basketball league
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q18536323 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
5


In [27]:
#return properties for basketball association
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q15986354 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
2


In [28]:
#return properties for professional sports league
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q15991290 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
5


In [29]:
#return what is Durant participant in
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29545 wdt:P1344 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q39080780'), ('name', "basketball at the 2020 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q18416654'), ('name', "basketball at the 2016 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q749036'), ('name', "basketball at the 2012 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8613'), ('name', '2016 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q211438'), ('name', '2010 FIBA World Championship')]
[('o', 'http://www.wikidata.org/entity/Q52702755'), ('name', '2018–19 NBA season')]
[('o', 'http://www.wikidata.org/ent

In [30]:
#return properties having basketball team as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q13393265 .
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('name', 'category contains')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
9


In [31]:
#return also the subject 
queryString = """
SELECT DISTINCT ?s ?sName ?p ?name 
WHERE {
   ?s ?p wd:Q13393265 .
   ?p sc:name ?name.
   ?s sc:name ?sName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8206968'), ('sName', 'Hebraica y Macabi'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q5144784'), ('sName', "Colgate Raiders men's basketball"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q7051115'), ('sName', "Norfolk State Spartans men's basketball"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q7397248'), ('sName', "Sacred Heart Pioneers men's basketball"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q15725839'), ('sName', 'A.S. Kentavros Neas Kallikrateias'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q107876527'), ('sName', "Iowa Western Reivers men's basketball"), ('p', 'http://www.wiki

In [32]:
#return properties for BC Zepter Vienna
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q810271 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q1741'), ('oName', 'Vienna')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q40'), ('oName', 'Austria')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13393265'), ('oName', 'basketball team')]
[('p', 'http://www.wikidata.org/prop/direct/P6112'), ('name', 'category for members of a team'), ('o', 'http://www.wikidata.org/entity/Q18710411'), ('oName', 'Category:BC Zepter Vienna players')]
5


In [33]:
#return countries for nodes instances of basketball team 
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   ?team wdt:P31 wd:Q13393265.
   ?team wdt:P17 ?country.
   ?country sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q786'), ('name', 'Dominican Republic')]
[('country', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('country', 'http://www.wikidata.org/entity/Q1029'), ('name', 'Mozambique')]
[('country', 'http://www.wikidata.org/entity/Q1037'), ('name', 'Rwanda')]
[('country', 'http://www.wikidata.org/entity/Q1246'), ('name', 'Kosovo')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany

In [34]:
#return what is instance of basketball team and has Italy linked by country Italy
queryString = """
SELECT DISTINCT ?team ?name 
WHERE {
   ?team wdt:P31 wd:Q13393265.
   ?team wdt:P17 wd:Q38.
   ?team sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q16531388'), ('name', 'Virtus Basket Ariano')]
[('team', 'http://www.wikidata.org/entity/Q16586535'), ('name', 'Pallacanestro Brindisi')]
[('team', 'http://www.wikidata.org/entity/Q3891951'), ('name', 'Pallacanestro Livorno')]
[('team', 'http://www.wikidata.org/entity/Q3093425'), ('name', 'Libertas Trogylos Basket')]
[('team', 'http://www.wikidata.org/entity/Q3515600'), ('name', 'Taranto Cras Basket')]
[('team', 'http://www.wikidata.org/entity/Q36231'), ('name', 'S.S. Felice Scandone')]
[('team', 'http://www.wikidata.org/entity/Q36251'), ('name', 'Pallacanestro Biella')]
[('team', 'http://www.wikidata.org/entity/Q36419'), ('name', 'Pallacanestro Reggiana')]
[('team', 'http://www.wikidata.org/entity/Q3967121'), ('name', 'RB Montecatini Terme')]
[('team', 'http://www.wikidata.org/entity/Q48969'), ('name', 'Scaligera Basket Verona')]
[('team', 'http://www.wikidata.org/entity/Q50196'), ('name', 'Andrea Costa Imola')]
[('team', 'http://www.w

In [35]:
#all properties for Olimpia Milano
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q36362 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5995'), ('name', 'kit supplier'), ('o', 'http://www.wikidata.org/entity/Q161906'), ('oName', 'Oakley, Inc.')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q490'), ('oName', 'Milan')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('o', 'http://www.wikidata.org/entity/Q490'), ('oName', 'Milan')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q604681'), ('oName', 'Mediolanum Forum')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/e

In [36]:
#all properties for basketball team
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q13393265 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


In [37]:
#all properties and objects for basketball team
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q13393265 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q3665646'), ('oName', 'basketball player')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q5137571'), ('oName', 'basketball coach')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q847017'), ('oName', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"),

In [38]:
#all properties and objects for sports club
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q847017 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4438121'), ('oName', 'sports organization')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q19467997'), ('oName', 'Template:Infobox athletics club')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q12973014'), ('oName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q988108'), ('oName', 'club')]
[('p

In [39]:
#all properties and objects for club
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q988108 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12047392'), ('oName', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q43229'), ('oName', 'organization')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q622425'), ('oName', 'nightclub')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q48204'), ('oName', 'voluntary organization')]
[('p', 'http://www.wikidata.org/prop/

In [40]:
#all properties and objects for organization
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q43229 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3500'), ('oName', 'Ringgold ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P213'), ('oName', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oName', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3549'), ('oName', 'Australian Company Number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1278'), ('oName', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1320'), ('oName', 'Open

In [41]:
#all properties and objects for organization limit increased
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q43229 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3500'), ('oName', 'Ringgold ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P213'), ('oName', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oName', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P3549'), ('oName', 'Australian Company Number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1278'), ('oName', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1320'), ('oName', 'Open

In [42]:
#all properties and objects for Virtus Roma
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q36462 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue'), ('o', 'http://www.wikidata.org/entity/Q119882'), ('oName', 'Palazzo dello Sport')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('o', 'http://www.wikidata.org/entity/Q220'), ('oName', 'Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q220'), ('oName', 'Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13393265'), ('oName', 'basketball team')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league'), ('o', 'http://www.wikidata.org/en

In [43]:
#all properties and objects for lega Basket serie A
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q929124 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('name', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q124'), ('oName', 'October')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3270632'), ('oName', 'national championship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q623109'), ('oName', 'sports league')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('name', 'league level above'), ('o', 'http://www.wikidata.org/entity/Q185982'), ('oName', 'EuroLeague')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer'), ('o', 'http://

In [44]:
#all properties and objects for Italian basketball league system
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q16987658 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1756477'), ('oName', 'league system')]
3


In [45]:
#all properties and objects for Italian basketball league system
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q1107166 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q2485448'), ('oName', 'sports governing body')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation'), ('o', 'http://www.wikidata.org/entity/Q164836'), ('oName', 'FIBA Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation'), ('o', 'http://www.wikidata.org/entity/Q46199'), ('oName', 'FIBA')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q220'), ('oName', 'Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P2541'), ('name', 'operating area'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P740'), ('name', 'location of formation'), ('o', 'htt

In [46]:
#all properties and objects for fiba
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q46199 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q69329'), ('oName', 'Mies')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q69329'), ('oName', 'Mies')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q253414'), ('oName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11422536'), ('oName', 'international sport governing body')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of'), ('o', 'http://www.wikidata.org/entity/Q1067053'), ('oName', 'Global Association of International Sports Federations')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15986354'), ('oName', '

In [47]:
#all properties and objects for Italian Basketball Federation
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q1107166 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q2485448'), ('oName', 'sports governing body')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation'), ('o', 'http://www.wikidata.org/entity/Q164836'), ('oName', 'FIBA Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation'), ('o', 'http://www.wikidata.org/entity/Q46199'), ('oName', 'FIBA')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q220'), ('oName', 'Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P2541'), ('name', 'operating area'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P740'), ('name', 'location of formation'), ('o', 'htt

In [48]:
#all subjects and properties linked to Olimpia Milano
queryString = """
SELECT DISTINCT ?s ?name ?p ?pName
WHERE {
   ?s ?p wd:Q36362.
   ?s sc:name ?name.
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q557954'), ('name', 'Dino Meneghin'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q3691740'), ('name', '1992–93 FIBA Korać Cup'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q15067056'), ('name', 'Lega Basket Serie A 2013–14'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q3559305'), ('name', "1970–71 FIBA European Cup Winner's Cup"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q30594476'), ('name', '2017–18 LBA season'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q12879867'), ('name', "1975–76 FIBA European Cup Winner's Cup"), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s'

In [49]:
#all properties linked to Olimpia Milano
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   ?s ?p wd:Q36362.
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('pName', 'occupant')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P6087'), ('pName', 'coach of sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pName', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P5138'), ('pName', 'season of club or team')]
8


In [50]:
#return nodes participant in Olimpia Milano
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P710 wd:Q36362.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q16499714'), ('name', '2014–15 Euroleague')]
1


In [51]:
#return season of club or team of Olimpia Milano
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P5138 wd:Q36362.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17637379'), ('name', '2013–14 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q18224258'), ('name', '2014–15 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q26898785'), ('name', '2015–16 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q27188921'), ('name', '1969–70 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881675'), ('name', '1957–58 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881676'), ('name', '1958–59 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881677'), ('name', '1959–60 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881683'), ('name', '1970–71 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881684'), ('name', '1971–72 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881685'), ('name', '1972–73 Olimpia Milano season')]
[('s', 'http://www.wikidata.org/entity/Q3881688'

In [52]:
#all properties of Olimpia Milano
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q36362 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P276')

In [53]:
#return object linking Olimpia Milano inception
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q36362 wdt:P571 ?o.
   ?o sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [54]:
#return literal linking Olimpia Milano inception without name
queryString = """
SELECT DISTINCT ?o 
WHERE {
   wd:Q36362 wdt:P571 ?o.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1936-01-01T00:00:00Z')]
1


In [55]:
#return datatype of the literal found
queryString = """
SELECT DISTINCT ?o datatype(?o)
WHERE {
   wd:Q36362 wdt:P571 ?o.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1936-01-01T00:00:00Z'), ('callret-1', 'http://www.w3.org/2001/XMLSchema#dateTime')]
1


In [56]:
#cast to gYear
queryString = """
SELECT DISTINCT ?o xsd:gYear(?o)
WHERE {
   wd:Q36362 wdt:P571 ?o.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1936-01-01T00:00:00Z'), ('callret-1', '1936-01-01Z')]
1


In [57]:
#cast with year()
queryString = """
SELECT DISTINCT ?o (year(?o) as ?foundationYear)
WHERE {
   wd:Q36362 wdt:P571 ?o.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', '1936-01-01T00:00:00Z'), ('foundationYear', '1936')]
1


In [58]:
#return team instances of basketball team with country Italy with inception year before 1945
queryString = """
SELECT DISTINCT ?team ?name ?inceptionYear
WHERE {
   ?team wdt:P31 wd:Q13393265.
   ?team wdt:P17 wd:Q38.
   ?team wdt:P571 ?inceptionDate.
   BIND(year(?inceptionDate) as ?inceptionYear)
   FILTER(?inceptionYear < 1945)
   ?team sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('name', 'Nuova Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('name', 'Fortitudo Pallacanestro Bologna'), ('inceptionYear', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('name', 'Olimpia Milano'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('name', 'Mens Sana Basket'), ('inceptionYear', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('name', 'Virtus Pallacanestro Bologna'), ('inceptionYear', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('name', 'Pallacanestro Cantù'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q7127695'), ('name', 'Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q3754192'), ('name', 'Fulgor Omegna'), ('inceptionYear', '1937')]
[('team', 'http://www.wikidata.org/entity/Q36572'), ('name', 'Reyer Venezia

Final query for this task

In [59]:
# Return all italian basketball teams born before 1945 
#(the result set must be a list of triples basketball team IRI, label and year of inception).
# write your final query
queryString = """
SELECT DISTINCT ?team ?name ?inceptionYear
WHERE {
   ?team wdt:P31 wd:Q13393265.
   ?team wdt:P17 wd:Q38.
   ?team wdt:P571 ?inceptionDate.
   BIND(year(?inceptionDate) as ?inceptionYear)
   FILTER(?inceptionYear < 1945)
   ?team sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('name', 'Nuova Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('name', 'Fortitudo Pallacanestro Bologna'), ('inceptionYear', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('name', 'Olimpia Milano'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('name', 'Mens Sana Basket'), ('inceptionYear', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('name', 'Virtus Pallacanestro Bologna'), ('inceptionYear', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('name', 'Pallacanestro Cantù'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q7127695'), ('name', 'Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q3754192'), ('name', 'Fulgor Omegna'), ('inceptionYear', '1937')]
[('team', 'http://www.wikidata.org/entity/Q36572'), ('name', 'Reyer Venezia

## Task 5
Identify the BGP for NBA finals

In [60]:
# write your queries

In [61]:
#return properties for NBA 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q155223 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('p', 'http://www.wikidata.org/prop

In [62]:
#return properties for NBA increased limit
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q155223 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('p', 'http://www.wikidata.org/prop

In [63]:
#return also objects
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q155223 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18536323'), ('oName', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q15986354'), ('oName', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q238240'), ('oName', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of'), ('o', 'http://www.wikidata.org/entity/Q2455323'), ('oName', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q810343'), ('oName', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends'), ('o', 'http://www.wikidata.org/entity/Q118'), ('oName', 'April')]
[('p', 'http://www.wikidata.org/prop/direct/P4794')

In [64]:
#return properties for Lakers
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q121783 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/p

In [65]:
#return NBA winner
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q155223 wdt:P1346 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
1


In [66]:
#return properties pointing to NBA 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q155223.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('name', 'defendant')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('name', 'record label')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), (

In [67]:
#return also subjects
queryString = """
SELECT DISTINCT ?s ?sName ?p ?name
WHERE {
   ?s ?p wd:Q155223.
   ?p sc:name ?name.
   ?s sc:name ?sName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15908865'), ('sName', 'Jordan Adams'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q16227014'), ('sName', 'Tyler Ennis'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q16233299'), ('sName', 'Sean Kilpatrick'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q16236541'), ('sName', 'Sindarius Thornwell'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q17623132'), ('sName', 'C. J. Williams'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q498348'), ('sName', 'Jimmy Butler'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('s', 'http://www.wikidata.org/entity/Q23302736'), ('sName', 'Joel Bolomboy'), ('p

In [68]:
#return nodes part of NBA
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P361 wd:Q155223.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q238240'), ('name', 'Eastern Conference')]
[('s', 'http://www.wikidata.org/entity/Q318389'), ('name', 'Western Conference')]
2


In [69]:
#return nodes having league NBA
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P118 wd:Q155223.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15908865'), ('name', 'Jordan Adams')]
[('s', 'http://www.wikidata.org/entity/Q16227014'), ('name', 'Tyler Ennis')]
[('s', 'http://www.wikidata.org/entity/Q16233299'), ('name', 'Sean Kilpatrick')]
[('s', 'http://www.wikidata.org/entity/Q16236541'), ('name', 'Sindarius Thornwell')]
[('s', 'http://www.wikidata.org/entity/Q17623132'), ('name', 'C. J. Williams')]
[('s', 'http://www.wikidata.org/entity/Q498348'), ('name', 'Jimmy Butler')]
[('s', 'http://www.wikidata.org/entity/Q23302736'), ('name', 'Joel Bolomboy')]
[('s', 'http://www.wikidata.org/entity/Q29166745'), ('name', 'P. J. Dozier')]
[('s', 'http://www.wikidata.org/entity/Q127125'), ('name', 'Mark Price')]
[('s', 'http://www.wikidata.org/entity/Q2351523'), ('name', 'Mitchell Wiggins')]
[('s', 'http://www.wikidata.org/entity/Q442623'), ('name', 'Al Jefferson')]
[('s', 'http://www.wikidata.org/entity/Q443724'), ('name', 'Marco Belinelli')]
[('s', 'http://www.wikidata.org/entity/Q448245')

In [70]:
#return properties for Lebron
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q36159 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [71]:
#return what is Lebron participant in 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q36159 wdt:P1344 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q241329'), ('name', '2006 FIBA World Championship')]
[('obj', 'http://www.wikidata.org/entity/Q178595'), ('name', '2007 FIBA Americas Championship')]
[('obj', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
5


In [72]:
#return what 2007 FIBA Americas Championship is instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q178595 wdt:P31 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
1


In [73]:
#return sports season's subclasses
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P279 wd:Q27020041 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7700500'), ('name', 'tennis tour')]
[('s', 'http://www.wikidata.org/entity/Q105445345'), ('name', 'edition of the Six Nations Championship')]
[('s', 'http://www.wikidata.org/entity/Q106644070'), ('name', 'edition of the Rugby Europe International Championship')]
[('s', 'http://www.wikidata.org/entity/Q106978496'), ('name', 'edition of the World Rugby Nations Cup')]
[('s', 'http://www.wikidata.org/entity/Q47358534'), ('name', 'tennis tour edition')]
[('s', 'http://www.wikidata.org/entity/Q106043413'), ('name', 'edition of the Rugby World Cup')]
[('s', 'http://www.wikidata.org/entity/Q47403752'), ('name', 'tennis tournament edition by gender')]
[('s', 'http://www.wikidata.org/entity/Q10509145'), ('name', 'regular season')]
[('s', 'http://www.wikidata.org/entity/Q5290181'), ('name', 'domestic association football season')]
[('s', 'http://www.wikidata.org/entity/Q47345468'), ('name', 'tennis tournament edition')]
[('s', 'http://www.wikidata.o

In [74]:
#return what is Lebron nominated for
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q36159 wdt:P1411 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q680221'), ('name', 'Laureus World Sports Award for Breakthrough of the Year')]
[('o', 'http://www.wikidata.org/entity/Q4896605'), ('name', 'Best Championship Performance ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q4896598'), ('name', 'Best Breakthrough Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q1808301'), ('name', 'Laureus World Sports Award for Sportsman of the Year')]
[('o', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for Athlete of the Century')]
[('o', 'http://www.wikidata.org/entity/Q4896724'), ('name', 'Best Male Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q943106'), ('name', 'Best NBA Player ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q4896782'), ('name', 'Best Play ESPY Award')]
8


In [75]:
#return properties for Lebron
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q36159 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [76]:
#return what Lebron is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q36159 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [77]:
#return Lebron occupation
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q36159 wdt:P106 ?o .
   ?o sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


In [78]:
#return properties for basketball player
ueryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q3665646 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


In [79]:
#return who drafted Lebron
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q36159 wdt:P647 ?obj .
   ?obj sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
1


In [80]:
#return properties for Cleveland Cavaliers
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q162990 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P286'), ('name', 'head coach')]
[('p', 'http://www.wikidata.org/pro

In [81]:
#return properties pointing to Lakers
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q121783 .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6291'), ('name', 'advertises')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('name', 'occupant')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P6087'), ('name', 'coach of sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name',

In [82]:
#return subject having property winner object lakers
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P1346 wd:Q121783 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('s', 'http://www.wikidata.org/entity/Q631240'), ('name', '1979–80 NBA season')]
[('s', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('s', 'http://www.wikidata.org/entity/Q4566514'), ('name', '1949 BAA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4566866'), ('name', '1950 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4575074'), ('name', '1972 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4586947'), ('name', "1991 McDonald's Championship")]
[('s', 'http://www.wikidata.org/entity/Q4598928'), ('name', '2001 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q281094'), ('name', '1949–50 NBA season')]
[('s', 'http://www.wikidata.org/entity/Q3072478'), ('name', '1985 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q3072482'), ('name', '1987 NBA World Champ

In [83]:
#return what 2000 NBA Finals is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q4597667 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q842375'), ('name', 'NBA Finals')]
1


Final query for this task

In [155]:
#Identify the BGP for NBA finals
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q4597667 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q842375'), ('name', 'NBA Finals')]
1


## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [85]:
#return team instances of basketball team having Lebron linked by some property
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s wdt:P31 wd:13393265.
   wd:Q36159 ?p ?s.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [86]:
#return team instances of basketball team having Lebron linked by some property
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s wdt:P31 wd:13393265.
   ?s ?p wd:Q36159.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [87]:
#return team instances of basketball team having Lebron linked by some property
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:13393265.
   ?s ?p wd:Q36159.
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [88]:
#return team instances of basketball team having Lebron linked by some property
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q13393265.
   ?s ?p wd:Q36159.
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [89]:
#return team instances of basketball team having Lebron linked by some property
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s wdt:P31 wd:Q13393265.
   ?s ?p wd:Q36159.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [90]:
#return team instances of basketball team
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q13393265.
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8206968'), ('name', 'Hebraica y Macabi')]
[('s', 'http://www.wikidata.org/entity/Q5144784'), ('name', "Colgate Raiders men's basketball")]
[('s', 'http://www.wikidata.org/entity/Q7051115'), ('name', "Norfolk State Spartans men's basketball")]
[('s', 'http://www.wikidata.org/entity/Q7397248'), ('name', "Sacred Heart Pioneers men's basketball")]
[('s', 'http://www.wikidata.org/entity/Q15725839'), ('name', 'A.S. Kentavros Neas Kallikrateias')]
[('s', 'http://www.wikidata.org/entity/Q107876527'), ('name', "Iowa Western Reivers men's basketball")]
[('s', 'http://www.wikidata.org/entity/Q15927936'), ('name', 'Westchester Knicks')]
[('s', 'http://www.wikidata.org/entity/Q810271'), ('name', 'BC Zepter Vienna')]
[('s', 'http://www.wikidata.org/entity/Q100320348'), ('name', "East Central Tigers men's basketball")]
[('s', 'http://www.wikidata.org/entity/Q16201949'), ('name', "DePaul Blue Demons women's basketball")]
[('s', 'http://www.wikidata.org/e

In [91]:
#return properties for NBA finals
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q842375 ?p ?o .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('name', 'official name')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('name', 'month of the year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), (

In [92]:
#return properties pointing to NBA finals
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q842375.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
3


In [93]:
#return instances of NBA finals
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q842375.
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q24264147'), ('name', '2016 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q29096468'), ('name', '2017 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q4566514'), ('name', '1949 BAA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4566866'), ('name', '1950 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4575074'), ('name', '1972 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q4588662'), ('name', '1993 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q4598928'), ('name', '2001 NBA Finals')]
[('s', 'http://www.wikidata.org/entity/Q3072443'), ('name', '1947 BAA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q3072451'), ('name', '1960 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q3072455'), ('name', '1961 NBA World Championship Series')]
[('s', 'http://www.wikidata.org/entity/Q3072460'), ('name', '1962 NBA World 

In [94]:
#return properties pointing to 2016 NBA Final
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q24264147.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
2


In [95]:
#return also subjects
queryString = """
SELECT DISTINCT ?s ?sName ?p ?name 
WHERE {
   ?s ?p wd:Q24264147.
   ?p sc:name ?name.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q29096468'), ('sName', '2017 NBA Finals'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('s', 'http://www.wikidata.org/entity/Q19877916'), ('sName', '2015 NBA Finals'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
2


In [96]:
#return properties for 2016 NBA Final
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q24264147 ?p ?s .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P6760'), ('name', 'Know Your Meme ID')]
10


In [97]:
#return what NBA Final is instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q842375 wdt:P31 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q15275719'), ('name', 'recurring event')]
1


In [98]:
#return what NBA Final is subclass of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q842375 wdt:P279 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1366722'), ('name', 'final')]
1


In [99]:
#return what Final is subclass of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q1366722 wdt:P279 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q106635272'), ('name', 'competition stage')]
[('o', 'http://www.wikidata.org/entity/Q16510064'), ('name', 'sporting event')]
[('o', 'http://www.wikidata.org/entity/Q476300'), ('name', 'competition')]
[('o', 'http://www.wikidata.org/entity/Q61685689'), ('name', 'tournament system')]
[('o', 'http://www.wikidata.org/entity/Q30013662'), ('name', 'last')]
5


In [100]:
#return properties for sporting event
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q16510064 ?p ?o .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P3827'), ('name', 'JSTOR topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3911'), ('name', 'STW Thesaurus for Economics ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5250'), ('name', 'IAB code')]

In [101]:
#return properties pointing to sporting event
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q16510064.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop

In [102]:
#return properties pointing to 2016 NBA Finals
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    ?s ?p wd:Q24264147.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
2


In [103]:
#return properties for 2011 NBA Finals
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1320332 ?p ?o .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
7


In [104]:
#return 2011 NBA Finals' winner
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q1320332 wdt:P1346 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q132893'), ('name', 'Dallas Mavericks')]
1


In [105]:
#return properties pointing to Durant
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q29545.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('name', 'sponsor')]
[('p', 'http://www.wikidata.org/prop/direct/P3279'), ('name', 'statistical leader')]
6


In [106]:
#return also subjects
queryString = """
SELECT DISTINCT ?s ?sName ?p ?name
WHERE {
   ?s ?p wd:Q29545.
   ?p sc:name ?name.
   ?s sc:name ?sName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1959927'), ('sName', '2012 NBA All-Star Game'), ('p', 'http://www.wikidata.org/prop/direct/P3279'), ('name', 'statistical leader')]
[('s', 'http://www.wikidata.org/entity/Q1641970'), ('sName', 'John R. Wooden Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q189240'), ('sName', 'Kevin Garnett'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('s', 'http://www.wikidata.org/entity/Q222047'), ('sName', 'NBA Most Valuable Player Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q31391'), ('sName', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q4896724'), ('sName', 'Best Male Athlete ESPY Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'),

In [107]:
#return properties and objects for Durant
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q29545 ?p ?o .
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q113619'), ('oName', '2012–13 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q16191100'), ('oName', '2014-15 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('o', 'http://www.wikidata.org/entity/Q180950'), ('oName', 'Oklahoma City Thunder')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q19862716'), ('oName', '2015–16 NBA season')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q320084'), ('oName', 'Kevin Durand')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q39080780'), ('oName', "basketball at the 2020 Summe

In [108]:
#return teams having Durant as member
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q29545 wdt:P54 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q180950'), ('name', 'Oklahoma City Thunder')]
[('o', 'http://www.wikidata.org/entity/Q235326'), ('name', 'Seattle SuperSonics')]
[('o', 'http://www.wikidata.org/entity/Q572134'), ('name', 'Brooklyn Nets')]
[('o', 'http://www.wikidata.org/entity/Q7707830'), ('name', "Texas Longhorns men's basketball")]
[('o', 'http://www.wikidata.org/entity/Q244847'), ('name', "United States men's national basketball team")]
[('o', 'http://www.wikidata.org/entity/Q157376'), ('name', 'Golden State Warriors')]
6


In [156]:
#return teams having Lebron as member
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q36159 wdt:P54 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('o', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('o', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
3


In [109]:
#return teams having Durant or Lebron as member  
queryString = """
SELECT ?team ?teamName WHERE {
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q29545 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
UNION
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q29545 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('teamName', 'Brooklyn Nets')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('teamName', "Texas Longhorns men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('teamName', "United States men's national basketball team")]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('teamName', 'Brooklyn Nets')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('teamName', "Texas Longhorns men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q24

In [110]:
#return distinct teams having Durant or Lebron as member  
queryString = """
SELECT DISTINCT ?team ?teamName WHERE {
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q29545 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
UNION
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q29545 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('teamName', 'Brooklyn Nets')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('teamName', "Texas Longhorns men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('teamName', "United States men's national basketball team")]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors')]
6


In [111]:
#CORRECT
queryString = """
SELECT DISTINCT ?team ?teamName WHERE {
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q36159 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
UNION
{
    SELECT DISTINCT ?team ?teamName 
    WHERE {
       wd:Q29545 wdt:P54 ?team .
       ?team sc:name ?teamName.
    }
}
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers')]
[('team', 'http://www.wikidata.org/entity/Q162990'), ('teamName', 'Cleveland Cavaliers')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('teamName', 'Brooklyn Nets')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('teamName', "Texas Longhorns men's basketball")]
[('team', 'http://www.wikidata.org/entity/Q244847'), ('teamName', "United States men's national basketball team")]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors')]
9


In [112]:
#return teams having Durant or Lebron as member  and how many NBA finals they won
queryString = """
SELECT DISTINCT ?team ?teamName (COUNT(?final) AS ?totalFinals) WHERE {
    ?team wdt:P1346 ?final.
    ?final wdt:P31 wd:Q842375.
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q36159 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
    UNION
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q29545 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
}
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [158]:
#return teams having Durant or Lebron as member  and how many NBA finals they won
queryString = """
SELECT ?team ?teamName (COUNT(?final) AS ?totalFinals) WHERE {
    ?final wdt:P1346 ?team .
    ?final wdt:P31 wd:Q842375.
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q36159 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
    UNION
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q29545 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors'), ('totalFinals', '4')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('totalFinals', '16')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics'), ('totalFinals', '1')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat'), ('totalFinals', '2')]
4


Final query for this task

In [159]:
# Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals 
# won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return 
# a number for each basketball team).
# write your final query
queryString = """
SELECT ?team ?teamName (COUNT(?final) AS ?totalFinals) WHERE {
    ?final wdt:P1346 ?team .
    ?final wdt:P31 wd:Q842375.
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q36159 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
    UNION
    {
        SELECT DISTINCT ?team ?teamName 
        WHERE {
           wd:Q29545 wdt:P54 ?team .
           ?team sc:name ?teamName.
        }
    }
}
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors'), ('totalFinals', '4')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('totalFinals', '16')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics'), ('totalFinals', '1')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat'), ('totalFinals', '2')]
4


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [115]:
# return NBA team's members 
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   ?s wdt:P54 ?team.
   ?team wdt:P118 wd:Q155223.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q504025'), ('name', 'Bill Bradley')]
[('s', 'http://www.wikidata.org/entity/Q319336'), ('name', 'Stephon Marbury')]
[('s', 'http://www.wikidata.org/entity/Q543566'), ('name', 'DerMarr Johnson')]
[('s', 'http://www.wikidata.org/entity/Q221341'), ('name', 'Jason Kidd')]
[('s', 'http://www.wikidata.org/entity/Q2557595'), ('name', 'Bruno Šundov')]
[('s', 'http://www.wikidata.org/entity/Q2562179'), ('name', 'Gene Shue')]
[('s', 'http://www.wikidata.org/entity/Q2565934'), ('name', 'Ernie Grunfeld')]
[('s', 'http://www.wikidata.org/entity/Q16910147'), ('name', 'Emmanuel Mudiay')]
[('s', 'http://www.wikidata.org/entity/Q1618075'), ('name', 'James Bailey')]
[('s', 'http://www.wikidata.org/entity/Q60118'), ('name', 'Danilo Gallinari')]
[('s', 'http://www.wikidata.org/entity/Q125060'), ('name', 'Sergio Rodríguez')]
[('s', 'http://www.wikidata.org/entity/Q129041'), ('name', 'Carmelo Anthony')]
[('s', 'http://www.wikidata.org/entity/Q731502'), ('name',

In [116]:
# return Durant's country of citizenship
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   wd:Q29545 wdt:P27 ?country.
   ?country sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
1


In [117]:
#return what United States of America is instance of
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q30 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")

x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1489259'), ('name', 'superpower')]
[('o', 'http://www.wikidata.org/entity/Q99541706'), ('name', 'historical unrecognized state')]
[('o', 'http://www.wikidata.org/entity/Q1520223'), ('name', 'constitutional republic')]
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q512187'), ('name', 'federal republic')]
[('o', 'http://www.wikidata.org/entity/Q5255892'), ('name', 'democratic republic')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
7


In [118]:
#return properties for country
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q6256 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equi

In [119]:
#return properties for United States of America
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q30 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1125'), ('name', 'Gini coefficient')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1188'), ('name', 'Dharma Drum Institute of Liberal Arts place ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S

In [120]:
#return United States of America's continent
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q30 wdt:P30 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('o', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
2


In [121]:
# return for each continent how many NBA players there are 
queryString = """
SELECT DISTINCT ?continent ?name (COUNT(?player) AS ?totPlayers)
WHERE {
   ?player wdt:P54 ?team;
       wdt:P27 ?country.
   ?country wdt:P30 ?continent.
   ?team wdt:P118 wd:Q155223.
   ?continent sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia'), ('totPlayers', '45')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('totPlayers', '796')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania'), ('totPlayers', '9381')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa'), ('totPlayers', '132')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America'), ('totPlayers', '80')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia'), ('totPlayers', '178')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America'), ('totPlayers', '9550')]
7


In [122]:
#return what Europe is instance of
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q46 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
[('o', 'http://www.wikidata.org/entity/Q2418896'), ('name', 'part of the world')]
2


In [123]:
# return for each continent how many NBA players there are 
queryString = """
SELECT DISTINCT ?continent ?name (COUNT(?player) AS ?totPlayers)
WHERE {
   ?player wdt:P54 ?team;
       wdt:P27 ?country.
   ?country wdt:P30 ?continent.
   ?continent wdt:P31 wd:Q5107.
   ?team wdt:P118 wd:Q155223.
   ?continent sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('totPlayers', '796')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania'), ('totPlayers', '9381')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa'), ('totPlayers', '132')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America'), ('totPlayers', '80')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia'), ('totPlayers', '178')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America'), ('totPlayers', '9550')]
6


Final query for this task

In [124]:
# write your final query
#Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one 
#of these teams at least once. Also consider the country of citizenship of the players with the related continent. 
#Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples 
#of continent IRI, labels and #players)
queryString = """
SELECT DISTINCT ?continent ?name (COUNT(?player) AS ?totPlayers)
WHERE {
   ?player wdt:P54 ?team;
       wdt:P27 ?country.
   ?country wdt:P30 ?continent.
   ?continent wdt:P31 wd:Q5107.
   ?team wdt:P118 wd:Q155223.
   ?continent sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe'), ('totPlayers', '796')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania'), ('totPlayers', '9381')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa'), ('totPlayers', '132')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America'), ('totPlayers', '80')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia'), ('totPlayers', '178')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America'), ('totPlayers', '9550')]
6


## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

In [125]:
# write your queries

In [126]:
#return Durant's award received
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29545 wdt:P166 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7865896'), ('name', 'USBWA National Freshman of the Year')]
[('o', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('o', 'http://www.wikidata.org/entity/Q1641970'), ('name', 'John R. Wooden Award')]
[('o', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q31391'), ('name', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for Athlete of the Century')]
[('o', 'http://www.wikidata.org/entity/Q4896724'), ('name', 'Best Male Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q644357'), ('name', 'NBA Rookie of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q674359'), ('name', 'All-NBA Team')]
[('o', 'http://www.wikidata.org/entity/Q943106'), ('name', 'Best NBA Player ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q4896727'), ('name'

Final query for this task

In [160]:
#Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)
# write your final query
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29545 wdt:P166 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7865896'), ('name', 'USBWA National Freshman of the Year')]
[('o', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('o', 'http://www.wikidata.org/entity/Q1641970'), ('name', 'John R. Wooden Award')]
[('o', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q31391'), ('name', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for Athlete of the Century')]
[('o', 'http://www.wikidata.org/entity/Q4896724'), ('name', 'Best Male Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q644357'), ('name', 'NBA Rookie of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q674359'), ('name', 'All-NBA Team')]
[('o', 'http://www.wikidata.org/entity/Q943106'), ('name', 'Best NBA Player ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q4896727'), ('name'

## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [128]:
#return nodes instances of NBA Most Valuable Player Award
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q222047.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [129]:
#return who have a link award received pointing to NBA Most Valuable Player Award
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P166 wd:Q222047.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('s', 'http://www.wikidata.org/entity/Q134183'), ('name', 'Magic Johnson')]
[('s', 'http://www.wikidata.org/entity/Q169452'), ('name', "Shaquille O'Neal")]
[('s', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson')]
[('s', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('s', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('s', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson')]
[('s', 'http://www.wikidata.org/entity/Q190152'), ('name', 'Larry Bird')]
[('s', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley')]
[('s', 'http://www.wikidata.org/entity/Q193020'), ('name', 'Tim Duncan')]
[('s', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook')]
[('s', 'http://www.wikidata.org/entity/Q199896'), ('name', 'Karl Malone')]
[('s', 'http://www.wikidata.org/entity/Q209921'), ('name'

In [130]:
#return who have a link award received pointing to NBA Most Valuable Player Award without distinct
queryString = """
SELECT ?s ?name
WHERE {
   ?s wdt:P166 wd:Q222047.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('s', 'http://www.wikidata.org/entity/Q134183'), ('name', 'Magic Johnson')]
[('s', 'http://www.wikidata.org/entity/Q169452'), ('name', "Shaquille O'Neal")]
[('s', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson')]
[('s', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('s', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('s', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson')]
[('s', 'http://www.wikidata.org/entity/Q190152'), ('name', 'Larry Bird')]
[('s', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley')]
[('s', 'http://www.wikidata.org/entity/Q193020'), ('name', 'Tim Duncan')]
[('s', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook')]
[('s', 'http://www.wikidata.org/entity/Q199896'), ('name', 'Karl Malone')]
[('s', 'http://www.wikidata.org/entity/Q209921'), ('name'

In [131]:
#return properties for NBA Most Valuable Player Award
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q222047 ?p ?o.
   
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct

In [132]:
#return Durant' awards received 
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q29545 wdt:P166 ?o .
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7865896'), ('name', 'USBWA National Freshman of the Year')]
[('o', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('o', 'http://www.wikidata.org/entity/Q1641970'), ('name', 'John R. Wooden Award')]
[('o', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q31391'), ('name', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('o', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for Athlete of the Century')]
[('o', 'http://www.wikidata.org/entity/Q4896724'), ('name', 'Best Male Athlete ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q644357'), ('name', 'NBA Rookie of the Year Award')]
[('o', 'http://www.wikidata.org/entity/Q674359'), ('name', 'All-NBA Team')]
[('o', 'http://www.wikidata.org/entity/Q943106'), ('name', 'Best NBA Player ESPY Award')]
[('o', 'http://www.wikidata.org/entity/Q4896727'), ('name'

In [13]:
#return properties and objects for NBA Most Valuable Player Award
queryString = """
SELECT ?p ?name ?o ?oName
WHERE {
   wd:Q222047 ?p ?o.
   
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q179051'), ('oName', 'Kareem Abdul-Jabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q134183'), ('oName', 'Magic Johnson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q17717'), ('oName', 'Oscar Robertson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'http://www.wikidata.org/entity/Q182455'), ('oName', 'Wilt Chamberlain')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('o', 'htt

In [134]:
#return properties for Bill Russell
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q242185 ?p ?o .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4560'), ('name', 'National Collegiate Basketball Hall of Fame ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4718'), ('name', 'Basketball-Reference.com NBA coach ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1315'), ('name', 'NLA Trove ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[(

In [135]:
#return #return properties for Bill Russell
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q242185 ?p ?o .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct/P4560')]
[('p', 'http://www.wikidata.org/prop/direct/P4718')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://www.wikidata.org/prop/direct/P1050')]
[('p', 'http://www.wikidata.org/prop/direct/P106')]
[('p', 'http://www.wikidata.org/prop/direct/P106')]
[('p', 'http://www.wikidata.org/prop/direct/P106')]
[('p', 'http://www.wikidata.org/prop/direct/P106')]
[('p', 'http://www.wikidata.org/prop/direct/P118')]
[('p', 'http://www.wikidata.org/prop/direct/P118')]
[('p', 'http://www.wikidata.org/prop/direct/P1263')]
[('p', 'http://www.wikidata.org/prop/direct/P1315')]
[('p', 'http://www.wikidata.org/prop/dir

In [136]:
#return properties linking NBA Most Valuable Player Award and Lebron 
queryString = """
SELECT ?p ?name
WHERE {
   wd:Q222047 ?p wd:Q36159.
   
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
1


In [137]:
#return subproperties for winner
queryString = """
SELECT ?s ?name
WHERE {
   ?s wdt:P1647 wdt:P1346.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [138]:
#return subproperties for winner
queryString = """
SELECT ?o ?name
WHERE {
   wdt:P1346 wdt:P1647 ?o.
   
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [139]:
#return subproperties for NBA Most Valuable Player Award
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q222047 wdt:P1647 ?o.
   
   ?o sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [140]:
#return properties for winner
queryString = """
SELECT ?p ?name
WHERE {
   wdt:P1346 ?p ?o.
   
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [141]:
#return properties pointing to winner
queryString = """
SELECT ?p ?name
WHERE {
    ?s ?p wdt:P1346.
   
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [142]:
#return Lebron award received and properties for these awards
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q36159 wdt:P166 ?o.
   ?o ?p ?something.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('name', 'name in kana')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('name', 'on focus list of Wikimedia project')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Free

In [143]:
#return Lebron award won and properties for these awards
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q36159 wdt:P1346 ?o.
   ?o ?p ?something.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [144]:
#return properties for the winner of NBA MVP
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q222047 wdt:P1346 ?o.
   ?o ?p ?something.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4557'), ('name', 'Indiana Basketball Hall of Fame ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4507'), ('name', 'California Sports Hall of Fame athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4560'), ('name', 'National Collegiate Basketball Hall of Fame ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4718'), ('name', 'Basketball-Reference.com NBA coach ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4367'), ('name', 'Louisiana Sports Hall of Fame ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4751'), ('name', 'Sports-Reference.com college basketball coach ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8777'), ('name', 'NCAA Statistics coach ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4720'), ('name', 'Basketball-Reference.com WNBA coach ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), (

In [145]:
#trying to see if it works
queryString = """
SELECT DISTINCT ?subject ?name
WHERE {
   ?subject ?s wd:Q222047.
   ?subject ?p wd:P1346.
   ?subject ?o wd:Q36159.
   ?subject sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [146]:
#wrong used instance of instead of occupation for basketball player
queryString = """
SELECT DISTINCT ?s ?name (COUNT( ?mvpAward) AS ?totMvpAward)
WHERE {
   ?s wdt:P31 wd:Q3665646.
   
   ?s sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q97254286'), ('name', 'An Komasawa'), ('totMvpAward', '0')]
[('s', 'http://www.wikidata.org/entity/Q16567223'), ('name', 'Jefimija Karakašević'), ('totMvpAward', '0')]
2


In [6]:
#return all subj with occupation basketball player who received an mvp award and count how many mvp award they received
queryString = """
SELECT DISTINCT ?s ?name (COUNT(?s) AS ?totMvpAward)
WHERE {
   ?s wdt:P106 wd:Q3665646.
   ?s wdt:P166 wd:Q222047.
   ?s sc:name ?name
}
GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q380515'), ('name', 'Willis Reed'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q192955'), ('name', 'Steve Nash'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q136678'), ('name', 'James Harden'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q25369'), ('name', 'Kobe Bryant'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q36159'), ('name', 'LeBron James'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q41421'), ('name', 'Michael Jordan'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q205326'), ('name', 'Derrick Rose'), ('totMvpAward', '1')]
[('s',

In [149]:
queryString = """
SELECT DISTINCT ?s #?name ?date #(COUNT(?date) AS ?totMvpAward)
WHERE {
   #?s wdt:P106 wd:Q3665646.
   
   #wd:Q222047 p:P1346 ?statement1.
   #?statement1 ps:P1346 ?s.
   ?s p:P585 ?date.
   #?s sc:name ?name
}
#GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [150]:
# P1346 winner P585 point in time P166 award received
queryString = """
SELECT ?o  #(COUNT(?date) AS ?totMvpAward) ?s ?name
WHERE {
   wd:Q36159 wdt:P166 ?o. #wd:Q222047.
   
}
#GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1501719')]
[('o', 'http://www.wikidata.org/entity/Q4896605')]
[('o', 'http://www.wikidata.org/entity/Q4896598')]
[('o', 'http://www.wikidata.org/entity/Q743122')]
[('o', 'http://www.wikidata.org/entity/Q1418009')]
[('o', 'http://www.wikidata.org/entity/Q1465181')]
[('o', 'http://www.wikidata.org/entity/Q1653207')]
[('o', 'http://www.wikidata.org/entity/Q222047')]
[('o', 'http://www.wikidata.org/entity/Q31391')]
[('o', 'http://www.wikidata.org/entity/Q4835465')]
[('o', 'http://www.wikidata.org/entity/Q4896724')]
[('o', 'http://www.wikidata.org/entity/Q644357')]
[('o', 'http://www.wikidata.org/entity/Q674359')]
[('o', 'http://www.wikidata.org/entity/Q739499')]
[('o', 'http://www.wikidata.org/entity/Q943106')]
[('o', 'http://www.wikidata.org/entity/Q7080855')]
[('o', 'http://www.wikidata.org/entity/Q2282428')]
17


In [151]:
# return what is instance of MVP Award
queryString = """
SELECT ?s  
WHERE {
   ?s wdt:P31 wd:Q25408040.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [9]:
# P1346 winner P585 point in time P166 award received
#return all the mvp award won by lebron with wikidata qualifiers 
queryString = """
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX p: <http://www.wikidata.org/prop/>

SELECT DISTINCT ?date
WHERE {
   wd:Q36159 wdt:P106 wd:Q222047;
           p:P166 ?awardStat.
   ?awardStat pq:P585 ?date;
              ps:P166 wd:Q36159.
}
#GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
# P1346 winner P585 point in time P166 award received
#try again with different sintax
queryString = """
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX p: <http://www.wikidata.org/prop/>
SELECT DISTINCT ?item
WHERE {
   ?item wdt:P106 wd:Q3665646; #wd:Q222047.
         p:P166 ?awardStat.
   ?awardStat ?p ?o .#ps:P166 wd:Q222047.
   #           ps:P166 wd:Q103360.
   #?award wdt:P585 ?time.
}
#GROUP BY ?s ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [2]:
#return what is Lebron participant in
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q36159 wdt:P1344 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q241329'), ('name', '2006 FIBA World Championship')]
[('o', 'http://www.wikidata.org/entity/Q178595'), ('name', '2007 FIBA Americas Championship')]
[('o', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
5


In [3]:
#return what is Durant participant in
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q29545 wdt:P1344 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('o', 'http://www.wikidata.org/entity/Q39080780'), ('name', "basketball at the 2020 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q18416654'), ('name', "basketball at the 2016 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q749036'), ('name', "basketball at the 2012 Summer Olympics – men's tournament")]
[('o', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8613'), ('name', '2016 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q211438'), ('name', '2010 FIBA World Championship')]
[('o', 'http://www.wikidata.org/entity/Q52702755'), ('name', '2018–19 NBA season')]
[('o', 'http://www.wikidata.org/ent

In [4]:
#return property for 2012–13 NBA season
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q113619 ?p ?o .
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 20
"""

print("Results")

Results


Final query for this task

In [11]:
# write your final query
# Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA 
# Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)
queryString = """
SELECT DISTINCT ?s ?name (COUNT(?s) AS ?totMvpAward)
WHERE {
   ?s wdt:P106 wd:Q3665646.
   ?s wdt:P166 wd:Q222047.
   ?s sc:name ?name
}
GROUP BY ?s ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q380515'), ('name', 'Willis Reed'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q192955'), ('name', 'Steve Nash'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q136678'), ('name', 'James Harden'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q25369'), ('name', 'Kobe Bryant'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q36159'), ('name', 'LeBron James'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q41421'), ('name', 'Michael Jordan'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook'), ('totMvpAward', '1')]
[('s', 'http://www.wikidata.org/entity/Q205326'), ('name', 'Derrick Rose'), ('totMvpAward', '1')]
[('s',